# SRGAN Inference code


In [ ]:
! git clone https://huggingface.co/spaces/thiemcun203/super-resolution

Cloning into 'super-resolution'...
remote: Enumerating objects: 489, done.
remote: Total 489 (delta 0), reused 0 (delta 0), pack-reused 489
Receiving objects: 100% (489/489), 6.71 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (187/187), done.
Filtering content: 100% (7/7), 468.59 MiB | 117.09 MiB/s, done.


In [ ]:
!pip install -r /content/super-resolution/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 778.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 20.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolve

In [ ]:
import time
import streamlit as st
import subprocess
import numpy as np
from PIL import Image
from io import BytesIO
import sys
sys.path.append('/content/super-resolution/models/SRGAN/')
from srgan import *


image = Image.open("/content/kelly kayseas.jpg").convert("RGB")
srgan_model = GeneratorResnet()
device = torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda')
srgan_model = torch.load('/content/super-resolution/models/SRGAN/srgan_checkpoint.pth', map_location=device)
enhanced_image = srgan_model.inference(image)
enhanced_image.save("/content/srgan_image.jpg")

In [ ]:
#Surveillance part

names = ['AC', 'BP', 'DT', 'LM', 'LS', 'OC']

for name in names:
  for i in range(1,3):
    device = torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda')
    srgan_model = GeneratorResnet()
    srgan_model = torch.load('/content/super-resolution/models/SRGAN/srgan_checkpoint.pth', map_location=device)

    image = Image.open(f"/content/images/{name}{i}.png").convert("RGB")

    if(device == torch.device('cpu')):
      enhanced_image = srgan_model.inference(image)

    elif(device == torch.device('cuda')):
      image = transforms.ToTensor()(image).unsqueeze(0).to(device)
      image = srgan_model.test(image).to('cpu')
      image = Image.fromarray((x.squeeze(0).permute(1, 2, 0).detach().numpy() * 255).astype('uint8'))
      enhanced_image = transforms.functional.adjust_brightness(image, 1.1)

    enhanced_image.save(f"/content/out/{name}{i}.png")